In [1]:
import numpy as np

In [2]:
# Note - acceleration and deceleration will be equal
BUS_LENGTH = 18.3 # https://en.wikipedia.org/wiki/Classic_(transit_bus)
BUS_ACCELERATION = 2 # https://hypertextbook.com/facts/2001/MeredithBarricella.shtml
CAR_LENGTH = 4.7 # https://en.wikipedia.org/wiki/Mid-size_car
CAR_ACCELERATION = 4 # https://hypertextbook.com/facts/2001/MeredithBarricella.shtml

## Define the roadway
NUM_LANES = 1
SPEED_LIMIT = 30 # Defines the max speed for all vehicles
LIGHT_LOCATIONS = [100, 500, 1200, 1400, 2000] # Vehicles enter at zero meters
LIGHT_DUTIES = [.5, .5, .5, .5, .5]
LIGHT_CYCLES = [60, 60, 60, 60, 60]
LIGHT_PHASES = [0, 0, 0, 0, 0]

In [3]:
class Light(object):
    '''
    Represents a traffic signal
    
    var: loc, where it is on the street
    var: duty, percent of cycle the signal is green
    var: cycle, duration of cycle in seconds
    var: phase, offset from time zero of cycle start, in seconds
    '''
    def __init__(self, loc, duty, cycle, phase):
        self.loc = loc
        self.duty = duty
        self.cycle = cycle
        self.phase = phase
        return
    
    def green(self, time):
        '''
        Returns true if the light is green at a given time
        '''
        if ((time-self.phase) % self.cycle) < (self.duty*self.cycle):
            return True
        else:
            return False

In [4]:
class Vehicle(object):
    '''
    Represents a vehicle on the road
    
    var: max_speed, the fastest a vehicle will go (m/s)
    var: acceleration, the acceleration (m/s^2)
    
    var: position, the position of the front of the vehicle
    var: lane, the lane the vehicle occupies
    
    var: duration, the time since the vehicle entered the roadway
    
    '''
    def __init__(self, max_speed, acceleration, lane, speed, position=0):
        '''
        Fixed and starting values
        '''
        self.max_speed = max_speed
        self.acceleration = acceleration
        
        self.lane = lane
        
        self.speed = speed
        self.position = position
        self.duration = 0
        
        return
        
    def step(self, lights, road, time_step):
        '''
        param: lights, list of Light objects on the roadway
        param: road, Road object that holds lanes of the road
        param: time_step, duration of time to simulate
        '''
        my_lane = road.lanes[self.lane]
        # vf^2 = vi^2 + 2*a*d, a is negative
        stopping_distance = (self.speed**2)/(2*self.acceleration)
        
        obstructed = my_lane.obstructed(self.position, stopping_distance)
        stopped = road.stopped(self.position, stopping_distance)
        
        ## TODO: incorporate lane changing
        if not (obstructed or stopped):
            self.accelerate(time_step)
        else:
            self.decelerate(time_step)
        return
    
    def accelerate(self, time_step):
        if self.speed < self.max_speed:
            self.speed += time_step*self.acceleration
            if self.speed > self.max_speed:
                self.speed = self.max_speed
        self.move(time_step)
        return
    
    def decelerate(self, time_step):
        if self.speed > 0:
            self.speed -= time_step*self.acceleration
            if self.speed < 0:
                self.speed = 0
        self.move(time_step)
        return
    
    def move(self, time_step):
        self.position += self.speed*time_step
        return